# カタカナ15文字を自動識別するモデルの構築
# 課題投稿用Notebook
* これは課題投稿用ファイルです。
* このsubmit_katakana.ipynbファイルを編集し、学習済みモデルなどの必要ファイルを同じフォルダにおき、そのフォルダをzipしたものを投稿してください。

In [2]:
import numpy as np
import glob
import os,sys
import util
import pickle
%load_ext autoreload
%autoreload 2
from common.model import Conv2DNN

def makedataset():
    """
    データセットをつくる関数です。
    自由に編集してください。
    """
    
    # 次の行は変更しないこと
    test_data = util.loaddata()
    
    # 以下は自由に編集しても構いません
    # 必要な前処理をここに記述してください  
    
    # 正規化
    test_data = test_data / test_data.max()
    test_data = test_data.astype('float32')

    # 配列形式変更
    # test_data = test_data.reshape(test_data.shape[0], 28, 28, 1)

    return test_data


def func_predict(test_data, test_label):
    """
    予測する関数
    data : 画像データ
    return loss, accuracy
    引数とreturn以外は、自由に編集してください    
    """
    # 以下を自由に編集してください
    X_test = test_data
    y_test = test_label
    
    # with open("katakana_model.pickle", "rb") as f:
    #     model = pickle.load(f)
    
    with open("katakana_model_hp_params.pickle", "rb") as f:
        hp_params = pickle.load(f)
    model = Conv2DNN(
        input_dim=test_data.shape[1:], 
        weight_init_std=0.01,
        **hp_params
    )
    
    i = 0
    with open("katakana_model_bn_params.pickle", "rb") as f:
        bn_params = pickle.load(f)
    with open("katakana_model_params.pickle", "rb") as f:
        params = pickle.load(f)
    for key in hp_params['layer_params'].keys():
        if (
            ('Conv' in key) or ('Affine' in key)
        ):
            # print(i, key)
            model.layers[key].W = params[f'W{i}']
            model.layers[key].b = params[f'b{i}']
            i += 1
        elif ('BatchNorm' in key):
            model.layers[key].gamma = params[f'gamma{i}']
            model.layers[key].beta = params[f'beta{i}']
            model.layers[key].moving_mean = bn_params[f'moving_mean{i}']
            model.layers[key].moving_var = bn_params[f'moving_var{i}']
            i += 1
            
    # calculate loss, accuracy
    accuracy = model.accuracy(X_test, y_test)
    loss = model.loss(X_test, y_test, train_flg=False)
    
    return loss, accuracy # 編集不可


def main():
    """
    編集しないでください。
    """
    # テスト用データをつくる
    test_data = makedataset()

    # 予測し精度を算出する
    util.accuracy(func_predict, test_data)
    
    return


if __name__=="__main__":
    main()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
0
0.019737854996387076 0.996
1
0.012871470912805225 0.998
2
0.011173169725965248 0.996
3
0.024698210491838672 0.994
4
0.006620246004680035 0.998
5
0.023287248247754102 0.998
Test loss: 0.016398033396571726
Test accuracy: 0.9966666666666667
